In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import pandas as pd
import numpy as np

# Some useful utilities

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

def z_clip(xs, b):
    return [min(x, b) for x in xs]

def clip(xs, upper, lower):
    return [max(min(x, upper), lower) for x in xs]

def your_code_here():
    return 1

def test(msg, value, expected):
    if value == expected:
        print(f"{msg}: {value}, as expected")
    else:
        print(f"{msg}: OH NO! Got {value}, but expected {expected}.")

In [3]:
adult_data = pd.read_csv("adult_with_pii.csv", parse_dates=['DOB'])

## Question 1 (5 points)
Consider the following dataframes and the query `count_join`.

In [6]:
df1 = pd.DataFrame(data={'k': [1, 1, 1], 'v1': ['a', 'b', 'c']})
df2 = pd.DataFrame(data={'k': [1, 1, 2], 'v2': ['C', 'D', 'E']})
display(df1)
display(df2)

def count_join(df1, df2):
    return pd.merge(df1, df2, left_on='k', right_on='k').shape[0]

count_join(df1, df2)

,k,v1
0,1,a
1,1,b
2,1,c


,k,v2
0,1,C
1,1,D
2,2,E


6

 What is the local sensitivity of `count_join(df1, df2)`, and why?
 
 *Hint*: argue in terms of the "worst possible change" to the true database (`df1` and `df2`).

- The sensitivity of `count_join(df1, df2)` is 6 because this is the sum of the size of each original dataset, thus it is the maximum value that `count_join()` can change by.

## Question 2 (20 points)

Consider the following definitions, which provide functions for computing the local sensitivity for a counting query over a join at an arbitrary distance $k$.

In [8]:
def max_freq(df):
    return int(df['k'].value_counts().to_frame().max())

def ls_count_join_k(df1, df2, k):
    return max(k + max_freq(df1), k + max_freq(df2))

Write code that uses the propose-test-release framework to release a differentially private answer to the query `count_join`. Your solution should return `False` when the test phase of propose-test-release fails.

*Hint*: use the definition of `ls_count_join_k` to compute the local sensitivity at distance $k$.

In [13]:
def ptr_count_join(df1, df2, b, epsilon, delta):
    
    df = pd.merge(df1, df2, left_on='k', right_on='k')
    k = max_freq(df)
    noisy_distance = laplace_mech(k, ls_count_join_k(df1, df2, k), epsilon)
    threshold = np.log(2/delta)/(2*epsilon)
    
    if noisy_distance >= threshold:
        return laplace_mech(df.shape[0], b, epsilon)
    else:
        return None

# Note: your answers will differ due to randomness
print(f"b = 1: {ptr_count_join(df1, df2, 1, 1, 1e-5)}")
print(f"b = 20: {ptr_count_join(df1, df2, 20, 1, 1e-5)}")
print(f"b = 50: {ptr_count_join(df1, df2, 50, 1, 1e-5)}")

b = 1: 6.1788783033551296
b = 20: 0.12311789458664002
b = 50: 29.479980541541142


## Question 3 (20 points)

Consider the query `capgain_mean`, which calculates the mean of the `Capital Gain` column of the input dataframe.

In [16]:
def capgain_mean(df):
    return np.mean(df['Capital Gain'])

capgain_mean(adult_data)

1077.6488437087312

Write code that implements sample-and-aggregate for the `capgain_mean` query.

*Hint*: use `np.array_split` to split `df` into chunks.

*Hint*: use `clip` (defined above) to clip results.

In [29]:
def saa_capgain_mean(df, k, upper, lower, epsilon):
    
    chunk_size = int(np.ceil(df.shape[0] / k))
    chunks = [df[i:i+chunk_size] for i in range(0,df.shape[0],chunk_size)]
    results = [capgain_mean(chunk) for chunk in chunks]
    clipped_results = clip(results, upper, lower)
    
    dp_answer = laplace_mech(np.mean(clipped_results), (upper-lower)/k, epsilon)
    return dp_answer
    

print(f"k = 20: {saa_capgain_mean(adult_data, 20, 10000, 0, 1)}")
print(f"k = 200: {saa_capgain_mean(adult_data, 200, 10000, 0, 1)}")
print(f"k = 2000: {saa_capgain_mean(adult_data, 2000, 10000, 0, 1)}")

k = 20: 663.1447856884279
k = 200: 1130.4863058419485
k = 2000: 1065.2674814295708


## Question 4 (10 points)

Run the tests in Question 3 several times and observe the accuracy for each value of $k$. What value of $k$ seems to work the best? Describe the tradeoff in setting a value for $k$.

- The value `k = 2000` seems to produce the most accurate answer. The tradeoff is that although a greater value of `k` means lower sensitivity, there are more chunks and thus, the likelihood of accurate answers is lower.

## Question 5 (5 points)

Do you think sample and aggregate would work well for join queries like `count_join`? Why, or why not?

- 'Sample and aggregate' should work well with a query like `count_join`; such a mechanism would be excellent for reducing the sensitivity of such a query.